# Business Insights: Chinook Record Store
In this project, we'll use SQL to query the Chinook Record Store database to answer important questions about their business. In the first cell, we connect to our database file.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%html
<style>
table {float:left}
</style>

## Available Tables in Database

In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ('table', 'view')

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Identifying Records to Add to the Store
The Chinook record store has just signed a deal with a new record label who specializes in artists from the USA. We need to pick the records from their label that we should purchase for the store (options are in the table below). We'll look at what genres sell the best in the USA to help us make a recommendation. 

| Artist Name | Genre |  
| :- | :-|  
| Regal | Hip-Hop |  
| Meteor and the Girls | Pop |  
| Red Tone | Punk |  
| Slim Jim Bites | Blues |  
| MPX | Electronic |  
| Old School Billy | Jazz |  

In [5]:
%%sql

WITH 
    USA_tracks_sold AS
                        (
                            SELECT 
                                il.track_id,
                                il.quantity tracks_sold
                            FROM invoice_line il
                            JOIN invoice i ON il.invoice_id = i.invoice_id
                            JOIN customer c ON i.customer_id = c.customer_id
                            WHERE c.country = 'USA'
                        )

SELECT 
    g.name genre,
    SUM(tracks_sold) num_tracks_sold,
    (CAST(SUM(tracks_sold) as float) / (SELECT SUM(tracks_sold) FROM USA_tracks_sold)) * 100 per_tracks_sold
FROM USA_tracks_sold u
JOIN track t ON u.track_id = t.track_id
JOIN genre g ON t.genre_id = g.genre_id
GROUP BY genre
ORDER BY Num_Tracks_Sold DESC
LIMIT 10

 * sqlite:///chinook.db
Done.


genre,num_tracks_sold,per_tracks_sold
Rock,561,53.37773549000951
Alternative & Punk,130,12.369172216936251
Metal,124,11.798287345385347
R&B/Soul,53,5.042816365366318
Blues,36,3.425309229305423
Alternative,35,3.3301617507136063
Pop,22,2.093244529019981
Latin,22,2.093244529019981
Hip Hop/Rap,20,1.9029495718363463
Jazz,14,1.3320647002854424


The above results show the number of tracks and percentage of tracks sold in the USA by genre. Based on the above, we should purchase the following albums:
* Red Tone - Punk
* Slim Jim Bites - Blues
* Meteor and the Girls - Pop

## Sales Rep Performance 

In [6]:
%%sql 

SELECT
    e.first_name || ' ' || e.last_name employee_name,
    e.title,
    e.hire_date,
    SUM(i.total) total_sales
FROM employee e
JOIN customer c ON e.employee_id = c.support_rep_id
JOIN invoice i ON c.customer_id = i.customer_id
GROUP BY c.support_rep_id
ORDER BY total_sales DESC

 * sqlite:///chinook.db
Done.


employee_name,title,hire_date,total_sales
Jane Peacock,Sales Support Agent,2017-04-01 00:00:00,1731.510000000004
Margaret Park,Sales Support Agent,2017-05-03 00:00:00,1584.0000000000032
Steve Johnson,Sales Support Agent,2017-10-17 00:00:00,1393.9200000000028


Jane has the highest sales. We can see the total sales seems to be negatively correlated with hire date. The employees who have been hired more recently have lower sales which we would expect as they don't have as much experience.

## Sales by Country
Now we want to look at customer count and sales metrics for the different countries. Countries with only one customer will be grouped as 'Other'.

In [7]:
%%sql

WITH country_or_other AS 
                      (
                        SELECT
                            CASE
                                WHEN (
                                        SELECT COUNT(*)
                                        FROM customer
                                        WHERE country = c.country
                                     ) = 1
                                THEN 'Other'
                                ELSE c.country
                            END country,
                            c.customer_id,
                            i.*
                        FROM customer c
                        JOIN invoice i ON c.customer_id = i.customer_id
                      )
    
SELECT
    country,
    customer_count,
    ROUND(total_sales,2) as total_sales,
    ROUND(avg_sale_per_customer,2) as avg_sale_per_customer,
    ROUND(avg_order_value,2) as avg_order_value
FROM (
        SELECT 
            country,
            COUNT(DISTINCT customer_id) customer_count,
            COUNT(DISTINCT invoice_id) order_count,
            SUM(total) total_sales,
            SUM(total) / COUNT(DISTINCT customer_id) avg_sale_per_customer,
            SUM(total) / COUNT(DISTINCT invoice_id) avg_order_value,
            CASE 
                WHEN country = 'Other' THEN 1
                ELSE 0
            END sort
        FROM country_or_other
        GROUP BY country
     )
ORDER BY sort, total_sales DESC

 * sqlite:///chinook.db
Done.


country,customer_count,total_sales,avg_sale_per_customer,avg_order_value
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.58,8.72
Other,15,1094.94,73.0,7.45


The United States has the highest customer count and is bringing in the most sales with \$1,040. This is almost double the next highest country which is Canada at \\$535. 

## Album vs Single Track Purchases

The Chinook store is setup in a way that allows customers to make purchases by either buying the whole album or a collection of one or more individual tracks. Management is considering changing their purchasing strategy to save money. The new strategy would be to purchase only the most popular tracks from each album from record companies, instead of purchasing every track.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [8]:
%%sql

WITH album_purchase AS 
                   (
                    SELECT
                        i.invoice_id, 
                        CASE 
                            WHEN 
                                (
                                 SELECT
                                     track_id
                                 FROM invoice_line
                                 WHERE invoice_id = i.invoice_id
                                 EXCEPT
                                 SELECT
                                    track_id
                                 FROM track 
                                 WHERE album_id = a.album_id
                                ) IS NULL

                                AND

                                (
                                 SELECT
                                    track_id
                                 FROM track 
                                 WHERE album_id = a.album_id
                                 EXCEPT
                                 SELECT
                                     track_id
                                 FROM invoice_line
                                 WHERE invoice_id = i.invoice_id
                                ) IS NULL
                            THEN 'Yes'
                            ELSE 'No'
                        END AS full_album


                    FROM invoice i
                    JOIN invoice_line il ON i.invoice_id = il.invoice_id
                    JOIN track t on il.track_id = t.track_id
                    JOIN album a on t.album_id = a.album_id 
                    GROUP BY i.invoice_id
                   )
    
SELECT 
    full_album album_purchase,
    COUNT(full_album) num_of_purchases,
    (CAST(COUNT(full_album) AS float) / (SELECT 
                                            COUNT(DISTINCT invoice_id)
                                        FROM invoice)
                                        ) * 100 per_of_purchases
FROM album_purchase
GROUP BY full_album

 * sqlite:///chinook.db
Done.


album_purchase,num_of_purchases,per_of_purchases
No,500,81.43322475570032
Yes,114,18.566775244299674


Over 80% of people purchasing from the store purchase individual tracks instead of albums. It appears that purchasing single tracks vs albums would be a good option for Chinook Record Store. They will be able to save money and still align with what majority of their customers purchasing practices.

## Tracks Purchased vs Tracks Available
Now we want to look at the number of tracks available in each genre vs the sale popularity. Is the range of tracks available in line with the types of tracks our customers are purchasing?

In [9]:
%%sql

WITH 
tracks_available AS 
                   (
                    SELECT
                        g.name genre,
                        COUNT(*) num_tracks
                    FROM track t
                    JOIN genre g on t.genre_id = g.genre_id
                    GROUP BY g.genre_id
                   ),
tracks_purchased AS
                   (
                    SELECT 
                        g.name genre,
                        COUNT(il.track_id) num_tracks
                    FROM invoice_line il 
                    JOIN track t ON il.track_id = t.track_id
                    JOIN genre g ON t.genre_id = g.genre_id
                    GROUP BY g.genre_id
                   )

SELECT 
    ta.genre,
    tp.num_tracks num_tracks_purchased,
    ta.num_tracks num_tracks_available,
    ROUND(CAST(ta.num_tracks AS float) / tp.num_tracks,2) available_to_purchased_ratio,
    ROUND((CAST(tp.num_tracks AS float) / (SELECT COUNT(track_id) FROM invoice_line)) * 100,2) per_tracks_purchased,
    ROUND((CAST(ta.num_tracks AS float) / (SELECT COUNT(*) FROM track)) * 100,2) per_tracks_available
FROM tracks_available ta
LEFT JOIN tracks_purchased tp ON ta.genre = tp.genre
GROUP BY ta.genre
ORDER BY num_tracks_purchased DESC

 * sqlite:///chinook.db
Done.


genre,num_tracks_purchased,num_tracks_available,available_to_purchased_ratio,per_tracks_purchased,per_tracks_available
Rock,2635,1297,0.49,55.39,37.03
Metal,619,374,0.6,13.01,10.68
Alternative & Punk,492,332,0.67,10.34,9.48
Latin,167,579,3.47,3.51,16.53
R&B/Soul,159,61,0.38,3.34,1.74
Blues,124,81,0.65,2.61,2.31
Jazz,121,130,1.07,2.54,3.71
Alternative,117,40,0.34,2.46,1.14
Easy Listening,74,24,0.32,1.56,0.69
Pop,63,48,0.76,1.32,1.37


Some of the columns of the above table are ambiguous: 
* <code>num_tracks_purchased</code> - total number of tracks purchased for each genre, can include multiple purchases of the same track
* <code>num_tracks_available</code> - total number of tracks available for each genre
* <code>available_to_purchased_ratio</code> - ratio of total number of tracks available for each genre to number of tracks purchased for each genre 
* <code>per_tracks_purchased</code> - percentage of total tracks purchased for each genre
* <code>per_tracks_available</code> - percentage of total tracks available for each genre

Judging by the results, we can see some areas for improvement. For example, there are several genres where we have a large amount of available tracks compared to how frequently tracks from that genre are purchased. Latin, Classical, and Regae are a few examples. Perhaps we could spend our money more wisely and purchase more tracks in the popular genres.

### Unique Tracks Purchased
Below we'll try to get an idea of whether there are a small number of tracks that make up the majority of total purchases per genre. We'll look at how many unique tracks have been purchased for each genre.

In [10]:
%%sql

SELECT 
    g.name,
    COUNT(DISTINCT il.track_id) unique_tracks_purchased,
    ROUND(CAST(COUNT(DISTINCT il.track_id) AS FLOAT) / (
                                                 SELECT
                                                    COUNT(*) num_tracks
                                                 FROM track t2
                                                 JOIN genre g2 on t2.genre_id = g2.genre_id
                                                 GROUP BY g2.genre_id
                                                 HAVING g2.name = g.name
                                                 ),2) per_tracks_purchased
FROM invoice_line il
JOIN track t ON il.track_id = t.track_id
JOIN genre g on t.genre_id = g.genre_id
GROUP BY t.genre_id
ORDER BY 2 DESC

 * sqlite:///chinook.db
Done.


name,unique_tracks_purchased,per_tracks_purchased
Rock,915,0.71
Metal,238,0.64
Alternative & Punk,176,0.53
Latin,119,0.21
Jazz,61,0.47
Blues,56,0.69
R&B/Soul,55,0.9
Alternative,34,0.85
Electronica/Dance,29,0.97
Pop,25,0.52


For some genres, 80-100% of the total tracks available have been purchased at least once and for others less than 20% of the total tracks available have been purchased at least once.

In [11]:
%%sql

SELECT 
    COUNT(DISTINCT il.track_id) unique_tracks_purchased,
    (SELECT COUNT(*) FROM track) tracks_available,
    ROUND(CAST(COUNT(DISTINCT il.track_id) AS FLOAT) / (SELECT COUNT(*) FROM track), 2) per_tracks_purchased
FROM invoice_line il

 * sqlite:///chinook.db
Done.


unique_tracks_purchased,tracks_available,per_tracks_purchased
1806,3503,0.52


There's definitely opportunities here for better track selection, we see above that 52% of the total tracks available have never been purchased.